# Get Last Model

## Init MLFlow

In [138]:
import mlflow.pyfunc
import mlflow.keras
import os
os.environ["AWS_ACCESS_KEY_ID"] = "mlflow"
os.environ["AWS_SECRET_ACCESS_KEY"] = "mlflow123"
os.environ["MLFLOW_S3_ENDPOINT_URL"] = f"http://mlflow-minio.mlflow.svc.cluster.local:9000/"
mlflow.set_tracking_uri("http://mlflow.mlflow.svc.cluster.local")
mlflow.set_experiment("bitcoin")

<Experiment: artifact_location='s3://mlflow/1', creation_time=None, experiment_id='1', last_update_time=None, lifecycle_stage='active', name='bitcoin', tags={}>

## Get Model, MLFlow Run, Params

In [139]:
from mlflow import MlflowClient

client = MlflowClient()
models = client.search_registered_models(filter_string="name = 'BitcoinForecast'")
model = models[-1]
version = model.latest_versions[-1]

run = client.get_run(version.run_id)
params = run.data.params

# Get offline features

In [162]:
import pandas as pd
from datetime import datetime
entity_df_btc = pd.DataFrame.from_dict(
    {
        "symbol": "BTC/USD",
        "event_timestamp": [item for item in pd.date_range(datetime.utcfromtimestamp(float(params["data_to"]) ), datetime.utcnow(), freq='5min')]
    }
)

entity_df_eth = pd.DataFrame.from_dict(
    {
        "symbol": "ETH/USD",
        "event_timestamp": [item for item in pd.date_range(datetime.utcfromtimestamp(float(params["data_to"]) ), datetime.utcnow(), freq='5min')]
    }
)

In [163]:
from feast import FeatureStore
feature_store = FeatureStore('../../demonstration/feature_store/feature_repo')  # Initialize the feature store

feature_service = feature_store.get_feature_service("crypto_stats")

In [164]:
training_df_btc = feature_store.get_historical_features(features=feature_service, entity_df=entity_df_btc).to_df()
training_df_eth = feature_store.get_historical_features(features=feature_service, entity_df=entity_df_eth).to_df()

In [175]:
training_df_btc = training_df_btc.set_index(pd.DatetimeIndex(training_df_btc['event_timestamp']))
training_df_eth = training_df_eth.set_index(pd.DatetimeIndex(training_df_eth['event_timestamp']))

In [176]:
df = training_df_btc.join(training_df_eth, lsuffix="_btc", rsuffix="_eth")

In [177]:
df

,symbol_btc,event_timestamp_btc,open_btc,high_btc,low_btc,close_btc,symbol_eth,event_timestamp_eth,open_eth,high_eth,low_eth,close_eth
event_timestamp,,,,,,,,,,,,
2022-10-06 09:45:00,BTC/USD,2022-10-06 09:45:00,20132.02,20135.74,20128.27,20129.62,ETH/USD,2022-10-06 09:45:00,1358.99,1359.05,1357.98,1357.98
2022-10-06 09:50:00,BTC/USD,2022-10-06 09:50:00,20132.02,20135.74,20128.27,20129.62,ETH/USD,2022-10-06 09:50:00,1358.99,1359.05,1357.98,1357.98
2022-10-06 09:55:00,BTC/USD,2022-10-06 09:55:00,20132.02,20135.74,20128.27,20129.62,ETH/USD,2022-10-06 09:55:00,1358.99,1359.05,1357.98,1357.98
2022-10-06 10:00:00,BTC/USD,2022-10-06 10:00:00,20132.02,20135.74,20128.27,20129.62,ETH/USD,2022-10-06 10:00:00,1358.99,1359.05,1357.98,1357.98
2022-10-06 10:05:00,BTC/USD,2022-10-06 10:05:00,20132.02,20135.74,20128.27,20129.62,ETH/USD,2022-10-06 10:05:00,1358.99,1359.05,1357.98,1357.98
2022-10-06 10:10:00,BTC/USD,2022-10-06 10:10:00,20132.02,20135.74,20128.27,20129.62,ETH/USD,2022-10-06 10:10:00,1358.99,1359.05,1357.98,1357.98
2022-10-06 10:15:00,BTC/USD,2022-10-06 10:15:00,20132.02,20135.74,20128.27,20129.62,ETH/USD,2022-10-06 10:15:00,1358.99,1359.05,1357.98,1357.98
2022-10-06 10:20:00,BTC/USD,2022-10-06 10:20:00,20132.02,20135.74,20128.27,20129.62,ETH/USD,2022-10-06 10:20:00,1358.99,1359.05,1357.98,1357.98
2022-10-06 10:25:00,BTC/USD,2022-10-06 10:25:00,20132.02,20135.74,20128.27,20129.62,ETH/USD,2022-10-06 10:25:00,1358.99,1359.05,1357.98,1357.98


In [178]:
df = df.dropna()
first = df.index[0].timestamp()
last = df.index[-1].timestamp()

In [179]:
df['y'] = df['close_btc'].shift(-1)
df = df[:-1]

In [181]:
df = df[["open_btc" ,"high_btc", "low_btc", "close_btc", "open_eth" ,"high_eth", "low_eth", "close_eth", "y"]].drop_duplicates() 

In [182]:
df

,open_btc,high_btc,low_btc,close_btc,open_eth,high_eth,low_eth,close_eth,y
event_timestamp,,,,,,,,,
2022-10-06 09:45:00,20132.02,20135.74,20128.27,20129.62,1358.99,1359.05,1357.98,1357.98,20129.62


In [95]:
from sklearn.model_selection import train_test_split
random_state = 42
train_size = 0.8
X = df[["open_btc" ,"high_btc", "low_btc", "close_btc", "open_eth" ,"high_eth", "low_eth", "close_eth"]]
y = df['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size, random_state=random_state)

# Stateful Training

## Get Keras Model

In [96]:
model = mlflow.keras.load_model(
    model_uri=f"models:/{version.name}/{version.version}"
)

/home/pierre/miniconda3/envs/realtime-mlops/lib/python3.10/random.py:370: DeprecationWarning: non-integer arguments to randrange() have been deprecated since Python 3.10 and will be removed in a subsequent version
  return self.randrange(a, b+1)


## Fit Keras Model

In [110]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [105]:
import numpy as np
import tensorflow as tf
import logging

np.random.seed(random_state)
tf.random.set_seed(random_state) 

logger = logging.getLogger(__name__)

In [113]:
from mlflow.models.signature import infer_signature

with mlflow.start_run():
    mlflow.tensorflow.autolog()
    
#    if not model:
        model = Sequential()
        model.add(Dense(32, input_shape=(X_train.shape[-1],), activation="relu", name="hidden_layer"))
        model.add(Dense(32))
        model.add(Dense(1))
        model.compile(loss="mse", optimizer="adam")
    
    model.fit(X_train, y_train, epochs=15, batch_size=100, validation_split=.2)
            
    # Evaluate the best model with testing data.
    y_hat = model.predict(X_test)
    (rmse, mae, r2) = eval_metrics(y_test, y_hat)
    mlflow.log_param("data_from", first)
    mlflow.log_param("data_to", last)
    mlflow.log_param("feature_view", "crypto_stats")
    mlflow.log_param("framework", "tensorflow")
    mlflow.log_param("version", tf.__version__)
    mlflow.log_param("random_state", random_state)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    mlflow.log_metric("mae", mae)
    lr = res = {"lr_" + str(key): val for key, val in model.optimizer.get_config().items()}
    mlflow.log_params(lr)
    
    model_signature = infer_signature(X_train, y_train)
    info = mlflow.keras.log_model(model, version.name, registered_model_name="BitcoinForecast", signature=model_signature)
    
    uri = mlflow.get_artifact_uri()




Epoch 1/15
1/1 [==============================] - 0s 82ms/step - loss: 3528.6882 - val_loss: 731.6401
Epoch 2/15
1/1 [==============================] - 0s 31ms/step - loss: 136.1423 - val_loss: 1834.9414
Epoch 3/15
1/1 [==============================] - 0s 64ms/step - loss: 1784.3511 - val_loss: 4933.7480
Epoch 4/15
1/1 [==============================] - 0s 37ms/step - loss: 5222.1240 - val_loss: 5308.2363
Epoch 5/15
1/1 [==============================] - 0s 52ms/step - loss: 5627.9116 - val_loss: 2619.0457
Epoch 6/15
1/1 [==============================] - 0s 34ms/step - loss: 2675.9783 - val_loss: 592.9324
Epoch 7/15
1/1 [==============================] - 0s 35ms/step - loss: 212.3004 - val_loss: 1695.6647
Epoch 8/15
1/1 [==============================] - 0s 37ms/step - loss: 872.9787 - val_loss: 4135.9771
Epoch 9/15
1/1 [==============================] - 0s 60ms/step - loss: 3024.7200 - val_loss: 4634.8525
Epoch 10/15
1/1 [==============================] - 0s 46ms/step - loss: 3479.0

/home/pierre/miniconda3/envs/realtime-mlops/lib/python3.10/site-packages/mlflow/keras.py:405: UserWarning: The pyfunc inference behavior of Keras models logged with signatures differs from the behavior of Keras models logged without signatures. Specifically, when a signature is present, passing a Pandas DataFrame as input to the pyfunc `predict()` API produces an `ndarray` (for single-output models) or a dictionary of `str -> ndarray`: (for multi-output models). In contrast, when a signature is *not* present, `predict()` produces a Pandas DataFrame output in response to a Pandas DataFrame input.
  warnings.warn(


INFO:tensorflow:Assets written to: /tmp/tmp0ql18qm2/model/data/model/assets


Registered model 'BitcoinForecast' already exists. Creating a new version of this model...
2022/10/06 10:36:07 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: BitcoinForecast, version 2
Created version '2' of model 'BitcoinForecast'.


NameError: name 'model_artifact_name' is not defined

In [116]:
storage_uri = f"{uri}/{version.name}"

In [117]:
storage_uri

's3://mlflow/1/0a2df9ab40bc432ebd45d6931bc485c0/artifacts/BitcoinForecast'

In [120]:
template = f"""
apiVersion: "serving.kserve.io/v1beta1"
kind: "InferenceService"
metadata:
  name: "bitcoin-forecast"
  namespace: default
  labels:
    networking.knative.dev/visibility: cluster-local
spec:
  predictor:
    serviceAccountName: sa-s3
    model:
      modelFormat:
        name: mlflow
      protocolVersion: v2
      storageUri: {storage_uri}
  transformer:
    logger: 
      mode: response
      url: http://kafka-broker-ingress.knative-eventing.svc.cluster.local/default/crypto-prediction
    containers:
    - image: piepra/feast-transformer:1.8
      name: btc-transfomer
      command:
      - "python"
      - "app/app.py"
      args:
      - --feast_serving_url
      - feast-feature-server.feast.svc.cluster.local:80
      - --entity_ids
      - "BTC/USD"
      - --feature_refs
      - "crypto_stats:open"
      - "crypto_stats:high"
      - "crypto_stats:low"
      - "crypto_stats:close"
      - --protocol
      - v2
"""

with open('../../demonstration/feast-kserve-transform/deployment/isvc.yaml', 'w+') as f:
    f.writelines(template)
    logging.info("wrote to demonstration/feast-kserve-transform/deployment/isvc.yaml")

In [122]:
!kubectl apply -f ../../demonstration/feast-kserve-transform/deployment/isvc.yaml

inferenceservice.serving.kserve.io/bitcoin-forecast configured
